# Implement models with PyTorch to allow for GPU cuda compatability

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [3]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # tain the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [7]:
train_model("csvs/ml_X.csv", "built_models/torch_ml.pth")

Epoch 1/50, Loss: 0.595880925655365
Epoch 2/50, Loss: 0.6860882639884949
Epoch 3/50, Loss: 0.6742780804634094
Epoch 4/50, Loss: 0.5493234992027283
Epoch 5/50, Loss: 0.47827422618865967
Epoch 6/50, Loss: 0.5941622257232666
Epoch 7/50, Loss: 0.632110595703125
Epoch 8/50, Loss: 0.5354063510894775
Epoch 9/50, Loss: 0.394734263420105
Epoch 10/50, Loss: 0.566523015499115
Epoch 11/50, Loss: 0.41741159558296204
Epoch 12/50, Loss: 0.37386369705200195
Epoch 13/50, Loss: 0.25384917855262756
Epoch 14/50, Loss: 0.5124058723449707
Epoch 15/50, Loss: 0.2984180450439453
Epoch 16/50, Loss: 0.16419744491577148
Epoch 17/50, Loss: 0.09299708902835846
Epoch 18/50, Loss: 0.2454405575990677
Epoch 19/50, Loss: 0.2231168895959854
Epoch 20/50, Loss: 0.24786895513534546
Epoch 21/50, Loss: 0.12357369810342789
Epoch 22/50, Loss: 0.08130253851413727
Epoch 23/50, Loss: 0.09198063611984253
Epoch 24/50, Loss: 0.02575889602303505
Epoch 25/50, Loss: 0.03758594021201134
Epoch 26/50, Loss: 0.18248756229877472
Epoch 27/50,

In [8]:
train_model("csvs/ou_X.csv", "built_models/torch_ou.pth")

Epoch 1/50, Loss: 0.6914739608764648
Epoch 2/50, Loss: 0.6801265478134155
Epoch 3/50, Loss: 0.6909040212631226
Epoch 4/50, Loss: 0.6873812079429626
Epoch 5/50, Loss: 0.7001709342002869
Epoch 6/50, Loss: 0.6900724172592163
Epoch 7/50, Loss: 0.6903168559074402
Epoch 8/50, Loss: 0.6939902305603027
Epoch 9/50, Loss: 0.6876978874206543
Epoch 10/50, Loss: 0.7043657302856445
Epoch 11/50, Loss: 0.703919529914856
Epoch 12/50, Loss: 0.6875346899032593
Epoch 13/50, Loss: 0.6807803511619568
Epoch 14/50, Loss: 0.6952847838401794
Epoch 15/50, Loss: 0.6966292262077332
Epoch 16/50, Loss: 0.6939157843589783
Epoch 17/50, Loss: 0.6940783262252808
Epoch 18/50, Loss: 0.6829316020011902
Epoch 19/50, Loss: 0.6936244368553162
Epoch 20/50, Loss: 0.6830918788909912
Epoch 21/50, Loss: 0.6960296034812927
Epoch 22/50, Loss: 0.7040531039237976
Epoch 23/50, Loss: 0.6894449591636658
Epoch 24/50, Loss: 0.6947833895683289
Epoch 25/50, Loss: 0.7014769315719604
Epoch 26/50, Loss: 0.6978058815002441
Epoch 27/50, Loss: 0.7

In [9]:
train_model("csvs/pl_X.csv", "built_models/torch_pl.pth")

Epoch 1/50, Loss: 0.6859039068222046
Epoch 2/50, Loss: 0.7012361288070679
Epoch 3/50, Loss: 0.6712605953216553
Epoch 4/50, Loss: 0.5585998296737671
Epoch 5/50, Loss: 0.5911567807197571
Epoch 6/50, Loss: 0.5956783294677734
Epoch 7/50, Loss: 0.6961216926574707
Epoch 8/50, Loss: 0.5911647081375122
Epoch 9/50, Loss: 0.6020587086677551
Epoch 10/50, Loss: 0.6540765762329102
Epoch 11/50, Loss: 0.6168641448020935
Epoch 12/50, Loss: 0.5773475766181946
Epoch 13/50, Loss: 0.6599305272102356
Epoch 14/50, Loss: 0.4833906888961792
Epoch 15/50, Loss: 0.6083524823188782
Epoch 16/50, Loss: 0.45393580198287964
Epoch 17/50, Loss: 0.34880396723747253
Epoch 18/50, Loss: 0.41135373711586
Epoch 19/50, Loss: 0.6540074944496155
Epoch 20/50, Loss: 0.20006607472896576
Epoch 21/50, Loss: 0.4152814447879791
Epoch 22/50, Loss: 0.3027019500732422
Epoch 23/50, Loss: 0.21781770884990692
Epoch 24/50, Loss: 0.51275235414505
Epoch 25/50, Loss: 0.3104405999183655
Epoch 26/50, Loss: 0.22708070278167725
Epoch 27/50, Loss: 0